In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install tensorflow
!pip install keras

In [4]:
!pip install imgaug
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-63o1smsh
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-63o1smsh
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=28014 sha256=b9441071fece45145f3099339f75abf0a51a6c619e6de64ffef7bf17da28a10b
  Stored in directory: /tmp/pip-ephem-wheel-cache-g8zswdov/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
Successfully built ImageDataAugmentor


In [15]:
import cv2
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.preprocessing import image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
from imgaug import augmenters as iaa
import imgaug as ia

train_path = '/kaggle/input/shopee-round-2-product-detection-challenge/train/train';
test_path = '/kaggle/input/shopee-round-2-product-detection-challenge/test'

def get_files(path, save_data):
    for i in os.listdir(path):
        if(os.path.isdir(path + "/" + i)):
            get_files(path + "/" + i, save_data)
        else:
            save_data.append(path + "/" + i)
#             if("(1).jpg" in i):
#                 os.remove(path + "/" + i)
#                 print(i)
#             else:
#                 save_data.append(path + "/" + i)

In [6]:
df = pd.read_csv('../input/shopee-round-2-product-detection-challenge/train.csv')
result = []
for i in range(len(df)):
  if df.category[i]<10:
    result.append('0'+str(df.category[i]))
  else:
    result.append(str(df.category[i]))

df.category = result
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,03
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,03
2,f6c172096818c5fab10ecae722840798.jpg,03
3,251ffd610399ac00fea7709c642676ee.jpg,03
4,73c7328b8eda399199fdedec6e4badaf.jpg,03


In [16]:
def load_data():
    """This function downloads, extracts, loads, normalizes and one-hot encodes Flower Photos dataset"""
    # get all classes for this dataset (types of flowers) excluding LICENSE file
    # roses = list(data_dir.glob('roses/*'))
    # 20% validation set 80% training set
    image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
    # make the training dataset generator
    train_data_gen = image_generator.flow_from_directory(directory=train_path + "/", batch_size=batch_size,
                                                        target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
                                                        shuffle=True, subset="training")
    # make the validation dataset generator
    test_data_gen = image_generator.flow_from_directory(directory=train_path + "/", batch_size=batch_size, 
                                                        target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
                                                        shuffle=True, subset="validation")
    return train_data_gen, test_data_gen

In [17]:
def create_model(input_shape):
    # load MobileNetV2
    model = NASNetLarge(input_shape=input_shape, weights = 'imagenet')
    # remove the last fully connected layer
    model.layers.pop()
    # freeze all the weights of the model except the last 4 layers
    for layer in model.layers[:-4]:
        layer.trainable = False
    # construct our own fully connected layer for classification
    output = Dense(42, activation="softmax")
    # connect that dense layer to the model
    output = output(model.layers[-1].output)
    model = Model(inputs=model.inputs, outputs=output)
    # print the summary of the model architecture
    model.summary()
    # training the model using rmsprop optimizer
    model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])
    return model

In [19]:
from keras.applications import MobileNetV2, DenseNet201, NASNetLarge
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import Dense
from keras import Model
IMAGE_SHAPE = (331, 331, 3)
epochs = 3
batch_size = 20
train_generator, validation_generator = load_data()
model = create_model(input_shape=(IMAGE_SHAPE))

# count number of steps per epoch
training_steps_per_epoch = np.ceil(train_generator.samples / batch_size)
validation_steps_per_epoch = np.ceil(validation_generator.samples / batch_size)

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.
359751680/359748576 [==============================] - 5s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (Non

                                                                 normal_add_5_12[0][0]            
__________________________________________________________________________________________________
activation_178 (Activation)     (None, 21, 21, 2016) 0           normal_concat_12[0][0]           
__________________________________________________________________________________________________
activation_177 (Activation)     (None, 21, 21, 2016) 0           normal_concat_11[0][0]           
__________________________________________________________________________________________________
reduction_conv_1_reduce_12 (Con (None, 21, 21, 672)  1354752     activation_178[0][0]             
__________________________________________________________________________________________________
adjust_conv_projection_reduce_1 (None, 21, 21, 672)  1354752     activation_177[0][0]             
__________________________________________________________________________________________________
reduction_

In [37]:
# train using the generators
from keras.callbacks import ReduceLROnPlateau
# model.load_weights('../input/output/model_NASNetLarge_v3.h5')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)
model.fit_generator(train_generator, steps_per_epoch=training_steps_per_epoch//2,
                    validation_data=validation_generator, validation_steps=validation_steps_per_epoch//5,
                    epochs=1, verbose=1, callbacks=[reduce_lr])

Epoch 1/1
2108/2108 [==============================] - 1355s 643ms/step - loss: 1.1122 - accuracy: 0.6994 - val_loss: 1.0738 - val_accuracy: 0.7021


In [38]:
model.save("model_NASNetLarge_v6.h5")

In [39]:
from keras.preprocessing import image
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(331, 331),grayscale=False)
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [40]:
result_set = pd.read_csv("../input/shopee-round-2-product-detection-challenge/test.csv")
image_name = result_set.filename
result = []
for i,name in enumerate(image_name):
    pred = model.predict(load_image(test_path+"/test/"+name))
    y_pred = [np.argmax(probas) for probas in pred]
    result.append(str(y_pred[0]).zfill(2))
result_set.category = result

In [41]:
result_set.to_csv("result_kaggle_v7.csv", index=False)